In [915]:
import pandas as pd
import numpy as np 
import altair as alt
import warnings 
import seaborn as sns
from scipy import stats
import requests
import os
#import census_key
warnings.filterwarnings('ignore')
from urllib.request import urlopen
import zipfile
import requests
import xlrd
import openpyxl
from openpyxl import load_workbook

#pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_columns", 330)

# Place level Data 

### Load in Place based ACS figures 

In [916]:
#seems like hawaii is getting dropped b/c all CDP 

In [917]:
#drop out NE states and PR


In [919]:
acs_2019 = pd.read_csv("raw_data/place_ACS_data_2019.csv",dtype={"Geo_FIPS":str}) #from social explorer 

acs_2019.rename(columns={"Geo_FIPS":"fips_place",
                     "ACS19_5yr_B19013001":"plc_med_income",
                     "ACS19_5yr_B25064001":"plc_med_rent",
                     "ACS19_5yr_B25012011":"plc_renter_child_households",
                     "ACS19_5yr_B25115021":"plc_renter_child_single_parent_households_male",
                     "ACS19_5yr_B25115024":"plc_renter_child_single_parent_households_female",
                        "SE_B13004_001":"plc_povpop",
                           "SE_B13004_002":"plc_belowpov",
                           "ACS19_5yr_B01003001":"plc_pop",
                           "ACS19_5yr_B25003001":"plc_households",
                           "ACS19_5yr_B25003002":"plc_owners",
                           "ACS19_5yr_B25003003":"plc_renters",
                           "ACS19_5yr_B03002003":"plc_white",
                            "ACS19_5yr_B03002004":"plc_black",
                           "ACS19_5yr_B03002006":"plc_asian",
                            "ACS19_5yr_B03002005": "plc_amin",
                           "ACS19_5yr_B03002007":"plc_nhpil",
                           "ACS19_5yr_B03002009":"plc_other",
                            "ACS19_5yr_B03002012":"plc_hispanic"
                              },inplace=True)
#add male and female headed households together 
acs_2019["plc_renter_child_single_parent_households"] = acs_2019['plc_renter_child_single_parent_households_male'] + acs_2019["plc_renter_child_single_parent_households_female"]
acs_2019["plc_pov"] = acs_2019["plc_belowpov"]/acs_2019["plc_povpop"]

In [920]:
acs_2019 = acs_2019[['fips_place',"Geo_NAME","Geo_STUSAB",'plc_pop',
'plc_households',
'plc_owners',
'plc_renters',
'plc_white',
'plc_black',
'plc_amin',
'plc_asian',
'plc_nhpil',
'plc_other',
'plc_hispanic',
'plc_povpop',
'plc_belowpov','plc_pov',
'plc_med_income',
'plc_renter_child_households',
'plc_med_rent',
'plc_renter_child_single_parent_households']]

In [921]:
#drop out NE states and PR
acs_2019 = acs_2019[~acs_2019.Geo_STUSAB.isin(["pr","ct", "me", "ma", "nh", "ri", "vt"])]
#drop CDPs 
acs_2019 = acs_2019[~acs_2019.Geo_NAME.str.contains("CDP")]
#drop places that dont have a population 
acs_2019 = acs_2019[acs_2019.plc_pop > 0 ]


In [922]:
acs_2019.fips_place.nunique()

19327

### Load in CBDG data from 2019 and 2020


In [923]:
cbdg_2019 = pd.read_csv("raw_data/cbdg_place_2019.csv",dtype={"fips_place":str,"FIPS":str})
cbdg_2020 = pd.read_csv("raw_data/cbdg_place_2020.csv",dtype={"fips_place":str,"FIPS":str})
cbdg_2019.dropna(subset=["fips_place","CDBG19","HOME19","ESG19","HOPWA19","HTF19"],inplace=True)#doing this just drops out NE money 
cbdg_2020.dropna(subset=["fips_place",'CDBG20', 'HOME20',
       'ESG20', 'CDBG_CV1', 'CDBG_CV2', 'CDBG_CV3', 'ESG_CV1', 'ESG_CV2'],inplace=True)
cbdg_2019["fips_place"]  = cbdg_2019["fips_place"].str.split(".").str[0].str.zfill(7)
cbdg_2020["fips_place"]  = cbdg_2020["fips_place"].str.split(".").str[0].str.zfill(7)
cbdg_2020["FIPS"]  = cbdg_2020["FIPS"].str.zfill(5)
cbdg_2019["FIPS"]  = cbdg_2019["state"].astype(str).str.zfill(2) + cbdg_2019["county"].astype(str).str.zfill(3)

cbdg_data_place = cbdg_2020.merge(cbdg_2019,on=["fips_place","cntyname","FIPS"],how="left")
# label entitlement places 
cbdg_data_place_entitled = cbdg_data_place[cbdg_data_place.CDBG19 > 0]
cbdg_data_place["Entitlement Community Flag"] = np.where(cbdg_data_place["fips_place"].isin(cbdg_data_place_entitled.fips_place.unique()),1,0)
cbdg_data_place.drop(columns=["cntyname","FIPS"],inplace=True)

In [924]:
place_merged_final = acs_2019.merge(cbdg_data_place,on="fips_place",how="left")


In [925]:
place_merged_final.rename(columns={"Entitlement Community Flag":"Entitlement_Community_Flag",
                       'CDBG20':"plc_CDBG20", 
                        'HOME20':"plc_HOME20",
                        'ESG20':"plc_ESG20",
                        'CDBG_CV1':"plc_CDBG_CV1", 
                        'CDBG_CV2':"plc_CDBG_CV2", 
                        'CDBG_CV3':"plc_CDBG_CV3", 
                        'ESG_CV1':"plc_ESG_CV1", 
                        'ESG_CV2':"plc_ESG_CV2",
                        'CDBG19':"plc_CDBG19", 
                        'HOME19':"plc_HOME19", 
                        'ESG19':"plc_ESG19", 
                        'HOPWA19':"plc_HOPWA19", 
                        'HTF19':"plc_HTF19",
                                  "Geo_STUSAB":"stab",
                                  "Geo_NAME":"plc_name"},inplace=True)

In [926]:
#Post merge column cleanup


def post_merge_col_name_fix(df):
    """
    Inputs:
        df (dataframe) - Dataframe post merge where the same named cols with _x and _y are identical in values

    This function looks at the typical naming conventions that occur during a merge with the same column names and fixes these inputs. 
    """
    for col in df.columns:
        df.rename(columns={col:col.replace("_x","")},inplace=True)
    
    df.drop(columns=[x for x in df.columns if "_y" in x],inplace=True)

    return df 

place_merged_final= post_merge_col_name_fix(place_merged_final)
place_merged_final.drop(columns=["Unnamed: 0"],inplace=True)
#place_merged_final.to_csv("CDBG_data_Place_level_10212021.csv")

In [927]:
place_merged_final.shape

(19334, 37)

In [928]:
print(erap1.Location)

0                  California
2         State of California
3            Local Government
4              Alameda County
5                Butte County
                ...          
573          Local Government
574                Ada County
575             City of Boise
576                      Guam
577    Territorial Government
Name: Location, Length: 528, dtype: object


In [929]:
### load in treasury ERAP data 
state_names = pd.read_csv("https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv")

erap1 = pd.read_excel("raw_data/November-ERA-Data.xlsx",sheet_name="ERA1 State & Local Reporting")
erap2 = pd.read_excel("raw_data/November-ERA-Data.xlsx",sheet_name="ERA2 State & Local Reporting")

def make_header(df):
    """
    do some basic cleanup of the header row 
    """
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    return df
erap1 = make_header(erap1)
erap2 = make_header(erap2)
erap1["State"] = ""
erap2["State"] = ""

excel_file_erap1 = 'raw_data/November-ERA-Data.xlsx' 
wb_erap1 = load_workbook(excel_file_erap1, data_only = True)
sh_erap1 = wb_erap1['ERA1 State & Local Reporting']

#need to add labels for states - here im looking at color changes as in 
#excel sheet each state is highlighted, so we can change label we want whenever, we index
#through and see a new color 
state_name = "California"
for i in range(erap1.shape[0]):
    if i == 0:
        continue
    color_in_hex =sh_erap1['A'+str(i)].fill.start_color.tint
    if color_in_hex > 0:
        state_name = sh_erap1['A'+str(i)].value
    erap1.loc[i, "State"] = state_name    
    
excel_file_erap2 = 'raw_data/November-ERA-Data.xlsx' 
wb_erap2 = load_workbook(excel_file_erap2, data_only = True)
sh_erap2 = wb_erap1['ERA2 State & Local Reporting']

state_name = "California"
for i in range(erap2.shape[0]):
    if i == 0:
        continue
    color_in_hex =sh_erap2['A'+str(i)].fill.start_color.tint
    if color_in_hex > 0:
        state_name = sh_erap2['A'+str(i)].value
    erap2.loc[i,"State"] = state_name   
erap1 = make_header(erap1)
erap2 = make_header(erap2)
erap1.rename(columns={"California":"State"},inplace=True)
erap2.rename(columns={"California":"State"},inplace=True)
erap1 = erap1[erap1.Location != "Grand Total"]
erap2 = erap2[erap2.Location != "Grand Total"]
erap1=erap1.dropna(axis=1, how='all')
erap2=erap2.dropna(axis=1, how='all')
#merge data and drop DC 
erap1 = erap1.merge(state_names,on="State")
erap2 = erap2.merge(state_names,on="State")
erap1 = erap1[erap1.Location != "State/DC"]
erap2 = erap2[erap2.Location != "State/DC"]

#sum up funds and number of houesholds funds disbursed to 
erap1["Total Number of Households ERAP1 (Direct Allocation)"] = erap1[[x for x in erap1 if "Total" in x]].sum(axis=1)
erap2["Total Number of Households ERAP2 (Direct Allocation)"] = erap2[[x for x in erap2 if "Total" in x]].sum(axis=1)
erap1["ERAP1_total_funds (Direct Allocation)"] = erap1[[x for x in erap1.columns if "Amount" in x]].sum(axis=1)
erap2["ERAP2_total_funds (Direct Allocation)"] = erap2[[x for x in erap2.columns if "Amount" in x]].sum(axis=1)
#flag footnote six and then strip out 
#6Local government recipient that has redirected or will redirect ERA 1 funds to their county or state government. only exits in erap1
erap1["ERAP1_redirected_funds"] = np.where(erap1.Location.str.contains("6"),1,0)
erap1.Location = erap1.Location.str.strip("6")

erap1.rename(columns={"Assistance to Households Expenditure Ratio as of November 305,7":"Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)"},inplace=True)
#need to calculate this for ERAP 2 because no longer provided- calcuate figure
#for ERAP2 using footnote 5 for ERAP 1 "5Calculated as the sum of Q1, April, May, June, July, August, September, October, and November expenditure for assistance to households divided by 90% of the ERA1 allocation amount."
erap2["Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)"] = erap2["ERAP2_total_funds (Direct Allocation)"]/ (erap2["Full ERA2 Allocation, Award, and Disbursement \n$ Millions4"] * 0.9)

total = erap1.merge(erap2,on=["Location","State","Abbreviation"],how="outer")

#keep only cols i want to merge onto the other 990 data
total_subset = total[["Location","ERAP1_total_funds (Direct Allocation)",'Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)',
      'Total Number of Households ERAP1 (Direct Allocation)','Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)',
      'ERAP2_total_funds (Direct Allocation)','Total Number of Households ERAP2 (Direct Allocation)',"ERAP1_redirected_funds","State","Abbreviation"]]

total_subset_merge = total_subset.rename(columns={"Location":"County"})

total_subset_merge["County Name"] = total_subset_merge["County"] +" "+ total_subset_merge["Abbreviation"]
total_subset_merge = total_subset_merge.drop(columns={"County","State","Abbreviation"})
#fix mis name 
total_subset_merge.replace({"County Name":{
                                         'Dekalb County GA':"DeKalb County GA"}},inplace=True)

#clean up and fix type of expenditure ratio 
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(str).str.strip("%")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(str).str.strip("%")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(str).str.strip("+")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(str).str.strip("+")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(float)
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(float)

In [930]:
#crosswalk counties to places 

county_place_crosswalk = pd.read_csv("~/Desktop/work/990_scraping/ethans_code/crosswalk_final/code/ACS_2019.csv",dtype={"fips_county":str,
                                                           "fips_place":str,"state":str,"county":str})
county_place_crosswalk["county_fips"] = county_place_crosswalk["state"] + county_place_crosswalk["county"] 
county_place_crosswalk = county_place_crosswalk[["fips_place","cntyname","county_fips"]]

In [931]:
place_merged_final = place_merged_final.merge(county_place_crosswalk,on="fips_place",how="left")

In [932]:
place_merged_final = place_merged_final.drop_duplicates()
place_merged_final.shape

(19328, 39)

In [933]:
### missing place/county mapping of newly incorporated places - manual fix 
missing_place_county = pd.read_csv("raw_data/missing_places_county_mapping.csv",dtype={"fips_place":str,
                                                                                      "County FIPS":str})
missing_place_county.fips_place = missing_place_county.fips_place.str.zfill(7)
missing_place_county["county_fips2"] = missing_place_county["County FIPS"].str.zfill(5)
missing_place_county = missing_place_county[["fips_place","county_fips2"]]

In [934]:
place_merged_final = place_merged_final.merge(missing_place_county,on="fips_place",how="left")
place_merged_final['county_fips'] = place_merged_final['county_fips'].where(place_merged_final['county_fips'].notnull(), place_merged_final['county_fips2'])
place_merged_final.drop(columns=["county_fips2"],inplace=True)
#this place moved back to being unincorporated 

place_merged_final = place_merged_final[place_merged_final.plc_name != "Mastic Beach village"]

In [935]:
#place_merged_final[place_merged_final.county_fips.isna()] manually mapped these places to counties. newly incorporated

In [936]:
manually_mapped = pd.read_csv("raw_data/manual_mapped_missing_places.csv")
manually_mapped.rename(columns={"county FIPS":"cntyname",
                               "place":"plc_name"},inplace=True)
manually_mapped = manually_mapped[["plc_name","cntyname"]]
#read in county name to county FIPS mapping 
get_fips = pd.read_csv("https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv",encoding = "ISO-8859-1",dtype={"fips":str})
get_fips = get_fips[["long_name","fips"]]
get_fips["fips"] = get_fips["fips"].str.zfill(5)
get_fips.rename(columns={"fips":"county_fips",
                        "long_name":"cntyname"},inplace=True)
get_fips.replace({"cntyname":{"DoÐa Ana County NM":"Doña Ana County NM"}},inplace=True)
manually_mapped = get_fips.merge(manually_mapped,how="left")
manually_mapped.cntyname= manually_mapped.cntyname.str.rsplit(" ",1).str[0]

In [937]:
place_merged_final = place_merged_final.merge(manually_mapped,on="plc_name",how="left")

In [938]:
place_merged_final['county_fips'] = place_merged_final['county_fips_x'].where(place_merged_final['county_fips_x'].notnull(), place_merged_final['county_fips_y'])
place_merged_final['cntyname'] = place_merged_final['cntyname_x'].where(place_merged_final['cntyname_x'].notnull(), place_merged_final['cntyname_y'])


In [939]:
place_merged_final = place_merged_final.drop(columns=["cntyname_x","cntyname_y","county_fips_y","county_fips_x"])

In [940]:
county_dict = {'01043':"Cullman County", 
#'02158':, 
'02105':"Hoonah-Angoon Census Area", 
'02198':"Prince of Wales-Hyder Census Area", 
'02195':"Petersburg Census Area", 
'02275':"Wrangell City and Borough", 
'08014':"Broomfield County",
'12086':"Miami-Dade County", 
'20155':"Reno County", 
'21155':"Marion County", 
#'46102':, 
#'49059':, 
'55101':"Racine County"}

place_merged_final['cntyname'] = place_merged_final['cntyname'].where(place_merged_final['cntyname'].notnull(), place_merged_final.county_fips.map(county_dict))

In [941]:
place_merged_final.fips_place.isna().sum()

0

In [942]:
erap_new = total_subset_merge[total_subset_merge["County Name"].str.contains("City")]
place_merged_final = place_merged_final.dropna(subset=["plc_name"])
place_merged_final["plc_name"] = place_merged_final["plc_name"].str.strip()
place_merged_final["plc_name"] = place_merged_final["plc_name"].str.title()
place_merged_final["plc_name"] = place_merged_final["plc_name"].str.replace("City","")
place_merged_final["plc_name"] = place_merged_final["plc_name"].str.replace("(Balance)","")
place_merged_final["stab"] = place_merged_final["stab"].str.strip()


place_merged_final.stab = place_merged_final.stab.str.upper()
place_merged_final["City Name"] = "City of " + place_merged_final["plc_name"] + place_merged_final["stab"]
erap_new.rename(columns={"County Name":"City Name"},inplace=True)
erap_new.replace({"City Name":{'City and County of Denver CO':"City of Denver CO",
                              'City and County of San Francisco CA':"City of San Francisco CA",
                              'Augusta-Richmond County Consolidated Government (Balance)':"Augusta-Richmond County Consolidated Government  (Balance)",
                              "City of Indianapolis (Balance) IN":"City of Indianapolis  (Balance) IN",
                              'City of Jersey City NJ':"City of Jersey NJ",
                              'City of Kansas City MO':"City of Kansas MO",
                              'City of Oklahoma City OK':"City of Oklahoma OK",
                              'City of Saint Paul MN':"City of St. Paul MN",
                              "City of Salt Lake City UT":"City of Salt Lake UT",
                              'Lafayette City Parish LA':"City of Lafayette LA",
                              "New York City NY":"City of New York NY",
                              "Salt Lake City UT":"City of Salt Lake UT"}},inplace=True)

place_merged_final.replace({"City Name":{"City of Indianapolis  ()IN":"City of Indianapolis IN",
                                        'City of Huntsville AL':"City Of Huntsville AL",
                                        'City of Spokane WA':'City Of Spokane WA',
                                        "City of Augusta-Richmond County Consolidated Government ()GA":"City of Augusta GA",
                                        "City of Boise  ID":"City of Boise ID",
                                        'City of Huntsville AR':'City of Huntsville AL',
                                        "City of Jersey  NJ":"City of Jersey NJ",
                                        'City of Kansas  KS':"City of Kansas MO",
                                        'City of Oklahoma  OK':"City of Oklahoma OK",
                                        'City of Salt Lake  UT':"City of Salt Lake UT",
                                        'City Of Spokane WA':'City of Spokane WA'}},inplace=True)

In [943]:
place_merged_final.plc_name

0            Abbeville 
1           Adamsville 
2          Addison Town
3            Akron Town
4            Alabaster 
              ...      
19322    Wamsutter Town
19323    Wheatland Town
19324          Worland 
19325       Wright Town
19326        Yoder Town
Name: plc_name, Length: 19327, dtype: object

In [944]:
place_merged_final = place_merged_final.merge(erap_new,on="City Name",how="left")

place_merged_final.drop(columns=["City Name","Unnamed: 0","Unnamed: 0.1"],inplace=True,errors="ignore")
place_merged_final = post_merge_col_name_fix(place_merged_final)
place_merged_final.rename(columns={'ERAP1_total_funds (Direct Allocation)':"plc_erap1_funds",
       'Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)':"plc_ratio_erap_1",
       'Total Number of Households ERAP1 (Direct Allocation)':"plc_hh_erap_1",
       'Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)':"plc_ratio_erap_2",
       'ERAP2_total_funds (Direct Allocation)':"plc_erap2_funds", 'Total Number of Households ERAP2 (Direct Allocation)':"plc_hh_erap_2"},inplace=True)

In [945]:
place_merged_final.drop_duplicates().shape

(19328, 46)

In [946]:
#set(erap_new["City Name"].unique()).difference(set(place_merged_final["City Name"].unique()))

### Add AMI data place 

In [947]:
#read in AMI data, create county level FIPS 
ami = pd.read_excel("raw_data/hud_ami_2019.xlsx",dtype={"fips2010":str})
ami.State = ami.State.astype(str).str.zfill(2)
ami.County = ami.County.astype(str).str.zfill(3)
ami["FIPS"] = ami.State + ami.County
ami = ami[["Metro_Area_Name","l50_4","FIPS"]]
#ami = ami.drop_duplicates(subset=["county14"])
#ami.rename(columns={"fips2010":"fips_place"},inplace=True)

In [948]:
additional_ami = pd.DataFrame(np.array([["Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",59250,"2501724960"],
                                        ["Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",59250,"2502156000"],
                                        ["Brockton, MA HUD Metro FMR Area",46250,"2502308130"]]),
                                       columns=['Metro_Area_Name', 'l50_4', "FIPS"])
ami = pd.concat([ami,additional_ami])
ami.l50_4 = ami.l50_4.astype(int)

In [949]:
#so finding out here than non unique county/AMI matches are bascially heavily in NE, and then a few spots in NY and MO
#so few for NY and MO, going to manually choose the major metros and drop from AMI, then the NE places will drop out in the
#match 
double_ami = ami.groupby('FIPS').filter(lambda x: len(x['l50_4'].unique()) > 1)
drop_from_AMI = ["2905599999","3608799998","3611999998"]
ami = ami[~ami.FIPS.isin(drop_from_AMI)]
ami = ami.drop_duplicates(subset=["FIPS"])
ami.rename(columns={"FIPS":"county_fips"},inplace=True)
place_merged_final = place_merged_final.merge(ami,on="county_fips",how="left")

In [950]:
place_merged_final = place_merged_final[['stab', 'cntyname',"county_fips",'plc_name', 'fips_place', 
         'plc_pop', 'plc_households', 'plc_owners', 'plc_renters',
                                         'plc_white', 'plc_black', 'plc_amin',
       'plc_asian', 'plc_nhpil',  'plc_hispanic', 'plc_other','plc_povpop',
       'plc_belowpov', 'plc_pov','plc_med_income',
       'plc_med_rent', 'plc_renter_child_households',
       'plc_renter_child_single_parent_households',
        'plc_CDBG20', 'plc_HOME20', 'plc_ESG20',
       'plc_CDBG_CV1', 'plc_CDBG_CV2', 'plc_CDBG_CV3', 'plc_ESG_CV1',
       'plc_ESG_CV2', 'plc_CDBG19', 'plc_HOME19', 'plc_ESG19', 'plc_HOPWA19',
       'plc_HTF19','plc_erap1_funds', 'plc_ratio_erap_1', 'plc_hh_erap_1','plc_erap2_funds',
       'plc_ratio_erap_2',  'plc_hh_erap_2', 'Entitlement_Community_Flag']]

In [951]:
place_merged_final.county_fips.isna().sum()

2

In [952]:
se_data = pd.read_csv("raw_data/place_income_data.csv",dtype={"Geo_FIPS":str})
se_data= se_data[["Geo_FIPS","Geo_NAME","Geo_STUSAB",
                'SE_A14001_001',
 'SE_A14001_002',
 'SE_A14001_003',
 'SE_A14001_004',
 'SE_A14001_005',
 'SE_A14001_006',
 'SE_A14001_007',
 'SE_A14001_008',
 'SE_A14001_009',
 'SE_A14001_010',
 'SE_A14001_011',
 'SE_A14001_012',
 'SE_A14001_013',
 'SE_A14001_014',
 'SE_A14001_015',
 'SE_A14001_016',
 'SE_A14001_017',
 'ACS19_5yr_B25118001',
 'ACS19_5yr_B25118002',
 'ACS19_5yr_B25118003',
 'ACS19_5yr_B25118004',
 'ACS19_5yr_B25118005',
 'ACS19_5yr_B25118006',
 'ACS19_5yr_B25118007',
 'ACS19_5yr_B25118008',
 'ACS19_5yr_B25118009',
 'ACS19_5yr_B25118010',
 'ACS19_5yr_B25118011',
 'ACS19_5yr_B25118012',
 'ACS19_5yr_B25118013',
 'ACS19_5yr_B25118014',
 'ACS19_5yr_B25118015',
 'ACS19_5yr_B25118016',
 'ACS19_5yr_B25118017',
 'ACS19_5yr_B25118018',
 'ACS19_5yr_B25118019',
 'ACS19_5yr_B25118020',
 'ACS19_5yr_B25118021',
 'ACS19_5yr_B25118022',
 'ACS19_5yr_B25118023',
 'ACS19_5yr_B25118024',
 'ACS19_5yr_B25118025']]

se_data.rename(columns={'SE_A14001_001':"Total_households_2019",
                       'SE_A14001_002':"Total_income_under_10k",
                       'SE_A14001_003':"Total_income_10k_15k", 
                       'SE_A14001_004':"Total_income_15k_20k", 
                       'SE_A14001_005':"Total_income_20k_25k", 
                       'SE_A14001_006':"Total_income_25k_30k",
                       'SE_A14001_007':"Total_income_30k_35k", 
                       'SE_A14001_008':"Total_income_35k_40k", 
                       'SE_A14001_009':"Total_income_40k_45k", 
                       'SE_A14001_010':"Total_income_45k_50k",
                       'SE_A14001_011':"Total_income_50k_60k", 
                       'SE_A14001_012':"Total_income_60k_75k", 
                       'SE_A14001_013':"Total_income_75k-100k", 
                       'SE_A14001_014':"Total_income_100k_125k",
                       'SE_A14001_015':"Total_income_125k_150k", 
                       'SE_A14001_016':"Total_income_150k_200k", 
                       'SE_A14001_017':"Total_income_200k+",
                       'ACS19_5yr_B25118014':"Total_renter_occupied_Households_2019", 
                       'ACS19_5yr_B25118015':"Renter_total_income_under_5k", 
                       'ACS19_5yr_B25118016':"Renter_total_income_5k_10k",
                       'ACS19_5yr_B25118017':"Renter_total_income_10k_15k", 
                       'ACS19_5yr_B25118018':"Renter_total_income_15k_20k", 
                       'ACS19_5yr_B25118019':"Renter_total_income_20k_25k",
                       'ACS19_5yr_B25118020':"Renter_total_income_25k_35k", 
                       'ACS19_5yr_B25118021':"Renter_total_income_35k_50k", 
                       'ACS19_5yr_B25118022':"Renter_total_income_50k_75k",
                       'ACS19_5yr_B25118023':"Renter_total_income_75k_100k", 
                       'ACS19_5yr_B25118024':"Renter_total_income_100k_150k", 
                       'ACS19_5yr_B25118025':"Renter_total_income_150k+"},inplace=True)
se_data.rename(columns={"Geo_FIPS":"fips_place"},inplace=True)
#se_data = se_data.merge(ami,on="fips_place",how="left")
#places_final = se_data.drop_duplicates(subset=["fips_place","Total_income_under_10k"])

In [953]:
place_merged_final = place_merged_final.merge(ami,on="county_fips",how="left")

In [954]:
place_county_merge = place_merged_final[["fips_place","l50_4"]]

In [955]:
#place_county_merge = place_merged_final[["fips_place","l50_4"]]
#se_data = se_data.merge(ami,on="fips_place",how="left")
se_data  = se_data.merge(place_county_merge,on="fips_place",how="left")#added left 

In [956]:
places_final_total = se_data[["fips_place",'l50_4', 'Total_income_under_10k',
       'Total_income_10k_15k', 'Total_income_15k_20k', 'Total_income_20k_25k',
       'Total_income_25k_30k', 'Total_income_30k_35k', 'Total_income_35k_40k',
       'Total_income_40k_45k', 'Total_income_45k_50k', 'Total_income_50k_60k',
       'Total_income_60k_75k', 'Total_income_75k-100k',
       'Total_income_100k_125k', 'Total_income_125k_150k',
       'Total_income_150k_200k', 'Total_income_200k+']]

places_final_rental = se_data[["fips_place",'l50_4','Renter_total_income_under_5k',
       'Renter_total_income_5k_10k', 'Renter_total_income_10k_15k',
       'Renter_total_income_15k_20k', 'Renter_total_income_20k_25k',
       'Renter_total_income_25k_35k', 'Renter_total_income_35k_50k',
       'Renter_total_income_50k_75k', 'Renter_total_income_75k_100k',
       'Renter_total_income_100k_150k', 'Renter_total_income_150k+']]

In [957]:
places_final_rental_melt=places_final_rental.melt(id_vars=["l50_4","fips_place"])
#second number 9,999
lower_dict_renter={'Renter_total_income_under_5k':0, 
'Renter_total_income_5k_10k':5000,
'Renter_total_income_10k_15k':10000, 
'Renter_total_income_15k_20k':15000,
'Renter_total_income_20k_25k':20000, 
'Renter_total_income_25k_35k':25000,
'Renter_total_income_35k_50k':35000, 
'Renter_total_income_50k_75k':50000,
'Renter_total_income_75k_100k':75000, 
'Renter_total_income_100k_150k':100000,
'Renter_total_income_150k+':150000}

upper_dict_renter={'Renter_total_income_under_5k':4999, 
'Renter_total_income_5k_10k':9999,
'Renter_total_income_10k_15k':14999, 
'Renter_total_income_15k_20k':19999,
'Renter_total_income_20k_25k':24999, 
'Renter_total_income_25k_35k':34999,
'Renter_total_income_35k_50k':49999, 
'Renter_total_income_50k_75k':74999,
'Renter_total_income_75k_100k':99999, 
'Renter_total_income_100k_150k':149990,
'Renter_total_income_150k+':200000}

def calc_bounds(l50_4,value,lower_bound,upper_bound):
    if l50_4 > upper_bound:
        return value
    if l50_4 <= upper_bound:
        if l50_4 > lower_bound:
            tmp = upper_bound - lower_bound
            tmp2 = np.abs(lower_bound - l50_4)
            return round((tmp2/tmp)*value)
        else:
            return 0
        
places_final_rental_melt["lower_bound"] =  places_final_rental_melt['variable'].map(lower_dict_renter)
places_final_rental_melt["upper_bound"] =  places_final_rental_melt['variable'].map(upper_dict_renter)
#calculate lower and upper bound 
places_final_rental_melt['Renters_Below_50%ami'] = places_final_rental_melt.apply(lambda x: calc_bounds(x.l50_4,x.value,x.lower_bound,x.upper_bound), axis=1)
#get total sum of rentersd below 50% AMI 
renter_counts = places_final_rental_melt[["fips_place","Renters_Below_50%ami"]].groupby("fips_place").sum().reset_index()
#create mapping of geocode to total number of renters 
places_final_total_renter = place_merged_final[["plc_renters","fips_place"]]
places_final_total_renter_grouped = places_final_total_renter.merge(renter_counts,on="fips_place",how="left")#added left 
places_final_total_renter_grouped["Percent_of_Renter_Households_Below_50%AMI"] = 100*(places_final_total_renter_grouped["Renters_Below_50%ami"]/places_final_total_renter_grouped["plc_renters"])

### now same thing for total number of households 


In [958]:
places_final_total_melt = places_final_total.melt(id_vars=['fips_place', 'l50_4'])
lower_dict_total = {'Total_income_under_10k':0, 
                    'Total_income_10k_15k':10000,
                    'Total_income_15k_20k':15000, 
                    'Total_income_20k_25k':20000, 
                    'Total_income_25k_30k':25000,
                    'Total_income_30k_35k':30000, 
                    'Total_income_35k_40k':35000, 
                    'Total_income_40k_45k':40000,
                    'Total_income_45k_50k':45000, 
                    'Total_income_50k_60k':50000, 
                    'Total_income_60k_75k':60000,
                    'Total_income_75k-100k':75000, 
                    'Total_income_100k_125k':100000,
                    'Total_income_125k_150k':125000, 
                    'Total_income_150k_200k':150000,
                    'Total_income_200k+':200000
    
}
upper_dict_total = {'Total_income_under_10k':9999, 
                    'Total_income_10k_15k':14999,
                    'Total_income_15k_20k':19999, 
                    'Total_income_20k_25k':24999, 
                    'Total_income_25k_30k':29999,
                    'Total_income_30k_35k':34999, 
                    'Total_income_35k_40k':39999, 
                    'Total_income_40k_45k':44999,
                    'Total_income_45k_50k':49999, 
                    'Total_income_50k_60k':59999, 
                    'Total_income_60k_75k':74999,
                    'Total_income_75k-100k':99999, 
                    'Total_income_100k_125k':124999,
                    'Total_income_125k_150k':149999, 
                    'Total_income_150k_200k':199999,
                    'Total_income_200k+':300000
    
}
places_final_total_melt["lower_bound"]= places_final_total_melt['variable'].map(lower_dict_total)
places_final_total_melt["upper_bound"]= places_final_total_melt['variable'].map(upper_dict_total)
#calculate lower and upper bound 
places_final_total_melt['Total_Households_Below_50%ami'] = places_final_total_melt.apply(lambda x: calc_bounds(x.l50_4,x.value,x.lower_bound,x.upper_bound), axis=1)
#get total sum of rentersd below 50% AMI 
total_counts = places_final_total_melt[["fips_place","Total_Households_Below_50%ami"]].groupby("fips_place").sum().reset_index()
#create mapping of geocode to total number of renters 
places_final_total = place_merged_final[["plc_households","fips_place"]]
places_final_total_grouped = total_counts.merge(places_final_total,on="fips_place")
places_final_total_grouped["Percent_of_Households_Below_50%AMI"] = 100*(places_final_total_grouped["Total_Households_Below_50%ami"]/places_final_total_grouped["plc_households"])

In [959]:
final_df = places_final_total_renter_grouped.merge(places_final_total_grouped,on="fips_place",how="left")#added left 

In [960]:
final_df = final_df[["fips_place","Total_Households_Below_50%ami","Renters_Below_50%ami"]]

In [961]:
final_df.rename(columns={"Total_Households_Below_50%ami":"plc_sub50_ami",
                    "Renters_Below_50%ami":"plc_sub50_ami_renter",
                        "Geo_FIPS":"fips_place"},inplace=True)

In [962]:
place_merged_final = place_merged_final.merge(final_df,on=["fips_place"],how="left")

In [963]:
place_merged_final.rename(columns={"Metro_Area_Name":"metro_area_name",
                  "l50_4":"50%AMI_Threshold"},inplace=True)

In [964]:
place_merged_final = place_merged_final[['stab','metro_area_name', 'cntyname', 'county_fips', 'plc_name', 'fips_place', 'plc_pop',
       'plc_households', 'plc_owners', 'plc_renters', 'plc_white', 'plc_black',
       'plc_amin', 'plc_asian', 'plc_nhpil', 'plc_hispanic', 'plc_other',
       'plc_povpop', 'plc_belowpov', 'plc_pov', 'plc_med_income',
       'plc_med_rent', 'plc_renter_child_households',
       'plc_renter_child_single_parent_households','plc_sub50_ami', 'plc_sub50_ami_renter',"50%AMI_Threshold",'plc_CDBG20', 'plc_HOME20',
       'plc_ESG20', 'plc_CDBG_CV1', 'plc_CDBG_CV2', 'plc_CDBG_CV3',
       'plc_ESG_CV1', 'plc_ESG_CV2', 'plc_CDBG19', 'plc_HOME19', 'plc_ESG19',
       'plc_HOPWA19', 'plc_HTF19', 'plc_erap1_funds', 'plc_ratio_erap_1',
       'plc_hh_erap_1', 'plc_erap2_funds', 'plc_ratio_erap_2', 'plc_hh_erap_2',
       'Entitlement_Community_Flag']]

In [984]:
#all of these except one are alaska census designated places 
set(acs_2019.fips_place.unique()).difference(place_merged_final["fips_place"].unique())

{'0200065',
 '0200870',
 '0201200',
 '0201420',
 '0201860',
 '0203440',
 '0203550',
 '0203880',
 '0204210',
 '0206520',
 '0206630',
 '0208740',
 '0212680',
 '0213230',
 '0214330',
 '0215430',
 '0216360',
 '0217410',
 '0217740',
 '0218620',
 '0218950',
 '0219060',
 '0220380',
 '0220970',
 '0221040',
 '0221810',
 '0222250',
 '0222910',
 '0226760',
 '0227530',
 '0227640',
 '0229180',
 '0229290',
 '0230060',
 '0230940',
 '0233030',
 '0233360',
 '0233470',
 '0233910',
 '0234350',
 '0234460',
 '0236770',
 '0237430',
 '0237650',
 '0240400',
 '0241720',
 '0241940',
 '0242050',
 '0242160',
 '0242380',
 '0245460',
 '0246010',
 '0246890',
 '0247000',
 '0247990',
 '0251180',
 '0252390',
 '0252720',
 '0253050',
 '0253710',
 '0253930',
 '0254150',
 '0254920',
 '0256350',
 '0256600',
 '0256680',
 '0259650',
 '0260750',
 '0261080',
 '0262510',
 '0264600',
 '0265590',
 '0265700',
 '0265800',
 '0266140',
 '0266360',
 '0266470',
 '0267460',
 '0267680',
 '0268670',
 '0268890',
 '0269770',
 '0272960',
 '02

In [987]:
#this place was unincorporated, so we good. 
acs_2019[acs_2019.fips_place == "3646085"]

,fips_place,Geo_NAME,Geo_STUSAB,plc_pop,plc_households,plc_owners,plc_renters,plc_white,plc_black,plc_amin,plc_asian,plc_nhpil,plc_other,plc_hispanic,plc_povpop,plc_belowpov,plc_pov,plc_med_income,plc_renter_child_households,plc_med_rent,plc_renter_child_single_parent_households
17459,3646085,Mastic Beach village,ny,12610,4254,3192,1062,9190,554,57,184,0,300,2309,12574,1769,0.14,72137.00,421,1829.00,435


In [969]:
place_merged_final = place_merged_final.drop_duplicates()
place_merged_final = place_merged_final.drop_duplicates(subset=["fips_place"])
#drop census areas 
place_merged_final.metro_area_name = place_merged_final.metro_area_name.fillna("None")
place_merged_final = place_merged_final[~place_merged_final.metro_area_name.str.contains("Census Area")]

#manually fix missing county name 
place_merged_final.at[15783,"cntyname"] = "Oglala Lakota County"
place_merged_final.at[17746,"cntyname"] = "Wasatch County"
#only two missing county names are places in census designated areas in Alaksa (Edna Bay and Whale pass) can drop 
place_merged_final.dropna(subset=["cntyname"],inplace=True)

In [979]:
place_merged_final.mean()

county_fips                                         inf
fips_place                                          inf
plc_pop                                        10307.34
plc_households                                  3854.89
plc_owners                                      2182.24
plc_renters                                     1672.65
plc_white                                       5567.37
plc_black                                       1491.81
plc_amin                                          53.62
plc_asian                                        655.82
plc_nhpil                                         16.59
plc_hispanic                                    2230.80
plc_other                                        265.17
plc_povpop                                     10031.15
plc_belowpov                                    1522.60
plc_pov                                            0.15
plc_med_income                                 55568.38
plc_med_rent                                    

In [989]:
place_merged_final[place_merged_final.plc_med_income.isna()]

,stab,metro_area_name,cntyname,county_fips,plc_name,fips_place,plc_pop,plc_households,plc_owners,plc_renters,plc_white,plc_black,plc_amin,plc_asian,plc_nhpil,plc_hispanic,plc_other,plc_povpop,plc_belowpov,plc_pov,plc_med_income,plc_med_rent,plc_renter_child_households,plc_renter_child_single_parent_households,plc_sub50_ami,plc_sub50_ami_renter,50%AMI_Threshold,plc_CDBG20,plc_HOME20,plc_ESG20,plc_CDBG_CV1,plc_CDBG_CV2,plc_CDBG_CV3,plc_ESG_CV1,plc_ESG_CV2,plc_CDBG19,plc_HOME19,plc_ESG19,plc_HOPWA19,plc_HTF19,plc_erap1_funds,plc_ratio_erap_1,plc_hh_erap_1,plc_erap2_funds,plc_ratio_erap_2,plc_hh_erap_2,Entitlement_Community_Flag
101,AL,"Chambers County, AL",Chambers County,01017,Cusseta Town,0119216,90,41,35,6,86,3,0,1,0,0,0,90,20,0.22,NaN,NaN,2,1,17.00,4.00,25750.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,AL,"Marengo County, AL",Marengo County,01091,Dayton Town,0119912,31,9,5,4,13,18,0,0,0,0,0,31,0,0.00,NaN,NaN,3,3,4.00,4.00,25750.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,AL,"Sumter County, AL",Sumter County,01119,Epes Town,0124256,614,227,211,16,4,610,0,0,0,0,0,614,65,0.11,NaN,525.00,9,9,125.00,10.00,25750.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,AL,"Marengo County, AL",Marengo County,01091,Faunsdale Town,0125816,23,16,6,10,16,7,0,0,0,0,0,23,9,0.39,NaN,NaN,1,0,10.00,7.00,25750.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,AL,"Tallapoosa County, AL",Tallapoosa County,01123,Goldville Town,0130448,52,26,20,6,50,0,0,0,0,2,0,52,7,0.13,NaN,NaN,0,0,10.00,6.00,26850.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18633,WV,"Boone County, WV HUD Metro FMR Area",Boone County,54005,Whitesville Town,5486836,377,171,116,55,370,0,0,0,0,0,7,374,157,0.42,NaN,456.00,7,10,91.00,49.00,27100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19278,WY,"Sweetwater County, WY",Sweetwater County,56037,Granger Town,5632870,40,26,24,2,36,0,0,0,0,4,0,40,12,0.30,NaN,NaN,0,0,16.00,2.00,44150.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19295,WY,"Converse County, WY",Converse County,56009,Lost Springs Town,5647805,1,1,1,0,1,0,0,0,0,0,0,1,0,0.00,NaN,NaN,0,0,1.00,0.00,41350.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19300,WY,"Niobrara County, WY",Niobrara County,56027,Manville Town,5650270,97,47,45,2,70,0,0,0,0,27,0,97,19,0.20,NaN,NaN,0,0,25.00,2.00,39850.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [991]:
acs_2019.plc_med_income

1       36875.00
2       46322.00
3       35625.00
4       25192.00
5       80072.00
          ...   
29308   70625.00
29312   60061.00
29316   52598.00
29317   80083.00
29318   38229.00
Name: plc_med_income, Length: 19327, dtype: float64

In [992]:
place_merged_final.shape

(19229, 47)

In [ ]:
place_merged_final.plc_med_income.

In [970]:
#only place missing 50%AMI_Threshold confirmed not in HUD data.
#for one place missing plc_pov data, confirmed missing in ACS data 
place_merged_final.isna().sum()

stab                                             0
metro_area_name                                  0
cntyname                                         0
county_fips                                      0
plc_name                                         0
fips_place                                       0
plc_pop                                          0
plc_households                                   0
plc_owners                                       0
plc_renters                                      0
plc_white                                        0
plc_black                                        0
plc_amin                                         0
plc_asian                                        0
plc_nhpil                                        0
plc_hispanic                                     0
plc_other                                        0
plc_povpop                                       0
plc_belowpov                                     0
plc_pov                        

In [978]:
place_merged_final.to_csv("processed_data/plc_ERAP_1_27_22.csv")

In [ ]:
#now just missing one fips place (NOW THERE ARE MORE )
set(acs_2019.fips_place.unique()).difference(set(place_merged_final.fips_place.unique()))

In [ ]:
#only one droped is one that become unicorporated! 
acs_2019[acs_2019.fips_place == '0220970']

In [ ]:
#place_merged_final.at[17834, 'cntyname'] = "Wasatch County"

In [ ]:
#so there are only 33 places we are missing, because they incorporated recenlty - need to readd data for all these palces 

In [ ]:
### now need to drop out new england 